In [14]:
import numpy as np
from tqdm.notebook import tqdm
# a = np.load("train_bone.npy")
a = np.load("train_joint.npy")
label = np.load("train_label.npy")
# c = np.load("train_bone_motion.npy")
print(a.shape)
# print(b.shape)
# print(c.shape)

(16432, 3, 300, 17, 2)


In [15]:
def get_random_samples(length, num_samples):
    data = np.array([i for i in range(length)])  # 数组包含 1 到 100 的数字
    random_samples = np.random.choice(data, size=num_samples, replace=False)
    return random_samples

temp_a = []
label_a = []

In [16]:
random_flipped_x = get_random_samples(a.shape[0], int(a.shape[0]*0.15))
# 水平x翻转
for i in random_flipped_x:
    temp = a[i, :, :, :, :]
    for t in range(a.shape[2]):
        for v in range(a.shape[3]):
            for m in range(a.shape[4]):
                temp[0,t,v,m] = -temp[0,t,v,m]
    temp_a.append(temp)
    label_a.append(label[i])

In [17]:
random_flipped_y = get_random_samples(a.shape[0], int(a.shape[0]*0.15))
# 水平y翻转
for i in random_flipped_y:
    temp = a[i, :, :, :, :]
    for t in range(a.shape[2]):
        for v in range(a.shape[3]):
            for m in range(a.shape[4]):
                temp[1,t,v,m] = -temp[1,t,v,m]
    temp_a.append(temp)
    label_a.append(label[i])

In [18]:
random_moved = get_random_samples(a.shape[0], int(a.shape[0]*0.15))
# 随机移动
for i in random_moved:
    temp = a[i, :, :, :, :]
    random_step = np.random.randint(-10, 10)
    for t in range(a.shape[2]):
        for v in range(a.shape[3]):
            for m in range(a.shape[4]):
                temp[0,t,v,m] = temp[0,t,v,m]+random_step
    temp_a.append(temp)
    label_a.append(label[i])

In [19]:
import numpy as np

def random_rotation_matrix():
    # 随机生成旋转角度
    theta_x = np.random.uniform(0, 1/6 * np.pi)  # X轴旋转
    theta_y = np.random.uniform(0, 1/6 * np.pi)  # Y轴旋转
    theta_z = np.random.uniform(0, 2 * np.pi)  # Z轴旋转

    # 计算旋转矩阵
    R_x = np.array([[1, 0, 0],
                    [0, np.cos(theta_x), -np.sin(theta_x)],
                    [0, np.sin(theta_x), np.cos(theta_x)]])
    
    R_y = np.array([[np.cos(theta_y), 0, np.sin(theta_y)],
                    [0, 1, 0],
                    [-np.sin(theta_y), 0, np.cos(theta_y)]])
    
    R_z = np.array([[np.cos(theta_z), -np.sin(theta_z), 0],
                    [np.sin(theta_z), np.cos(theta_z), 0],
                    [0, 0, 1]])
    
    # 合成旋转矩阵 (先绕 Z，后绕 Y，最后绕 X)
    R = R_z @ R_y @ R_x
    return R

def rotate_points(points,rotation_matrix):
    # rotation_matrix = random_rotation_matrix()
    # 对每个点应用旋转矩阵
    rotated_points = points @ rotation_matrix.T  # 矩阵乘法
    return rotated_points

# 示例三维坐标点数组
points = np.array(
    [1.0, 2.0, 3.0]
)

# 进行随机旋转
rotated_points = rotate_points(points,rotation_matrix=random_rotation_matrix())

print("原始坐标：")
print(points)
print("随机旋转后的坐标：")
print(rotated_points)

原始坐标：
[1. 2. 3.]
随机旋转后的坐标：
[ 2.6305972  -0.37064273  2.63487805]


In [20]:
random_rotate = get_random_samples(a.shape[0], int(a.shape[0]*0.3))
# 随机旋转
for i in random_rotate:
    temp = a[i, :, :, :, :]
    R = random_rotation_matrix()
    for t in range(a.shape[2]):
        for v in range(a.shape[3]):
            for m in range(a.shape[4]):
                temp[:,t,v,m] = rotate_points(temp[:,t,v,m],R)
    temp_a.append(temp)
    label_a.append(label[i])

In [21]:
temp_a = np.stack(temp_a)
label_b = np.stack(label_a)
print(temp_a.shape)
print(a.shape)
# temp_b = [a,temp_a]
temp_c = np.concatenate((a,temp_a))
label_c = np.concatenate((label,label_b))
print(temp_c.shape)
print(label_c.shape)
np.save("train_joint_augmented.npy", temp_c)
np.save("train_label_augmented.npy", label_c)

(12321, 3, 300, 17, 2)
(16432, 3, 300, 17, 2)
(28753, 3, 300, 17, 2)
(28753,)


In [22]:
# maxn = np.ones(a.shape[1]) 
# for n in tqdm(range(0, a.shape[0])):
#     for t in range(a.shape[2]):
#         for v in range(a.shape[3]):
#             for m in range(a.shape[4]):
#                 maxn = np.maximum(maxn, np.abs(a[n, :, t, v, m]))
#                 # print(a[n, :,t, v, m])
#                 # next_continue = input("Continue? (y/n): ")
# print(maxn)